# setting up classifiers

SCIENCE
thrashold: 0.4
------------------------
SPORTS
thrashold: 0.4
------------------------
POLITICS
thrashold: 0.5
------------------------
LIFESTYLE
thrashold: 0.5
------------------------
HEALTH
thrashold: 0.5
------------------------
EDUCATION
thrashold: 0.4
------------------------
WORLDPOST
thrashold: 0.4
------------------------
IMPACT
thrashold: 0.5
------------------------
BUSINESS
thrashold: 0.4
------------------------
RANDOM
thrashold: 0.5
------------------------

In [1]:
from access_data import *
from preparing_data import *
from evaluating_data import *
from feature_engineering import *

C:\Users\Max\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
#getting all data
df_huff= only_relevant_columns(dropping_new_category(combine_huff_categories(get_huffpost())))

# df_huff = df_huff[df_huff.category != 'RANDOM']
# df_huff = df_huff[df_huff.category != 'LIFESTYLE'] 

df_20news = get_20news()

df_all = df_huff
df_all = df_all.append(df_20news)

df_relevant = add_word_vector_column(df_all)
df_relevant = delete_short_datasets(df_all)

# getting single label
df_science = get_test_data_for_label(df_all.copy(), 'SCIENCE', 0.3)
df_sport = get_test_data_for_label(df_all.copy(), 'SPORTS', 0.3)
df_politics = get_test_data_for_label(df_all.copy(), 'POLITICS', 0.3)
df_lifestyle = get_test_data_for_label(df_all.copy(), 'LIFESTYLE', 0.3)
df_health = get_test_data_for_label(df_all.copy(), 'HEALTH', 0.3)
df_education = get_test_data_for_label(df_all.copy(), 'EDUCATION', 0.3)
df_worldpost = get_test_data_for_label(df_all.copy(), 'WORLDPOST', 0.3)
df_impact = get_test_data_for_label(df_all.copy(), 'IMPACT', 0.3)
df_business = get_test_data_for_label(df_all.copy(), 'BUSINESS', 0.3)
df_random = get_test_data_for_label(df_all.copy(), 'RANDOM', 0.3)

datasets = {"SCIENCE": df_science, "SPORTS": df_sport, "POLITICS": df_politics, "LIFESTYLE": df_lifestyle, "HEALTH": df_health, "EDUCATION": df_education, "WORLDPOST": df_worldpost, "IMPACT": df_impact, "BUSINESS": df_business, "RANDOM": df_random}

# from sklearn.decomposition import TruncatedSVD
# svd = TruncatedSVD(n_components=2500, n_iter=4, random_state=42)    # there are 52120 features

C:\Users\Max\Project\Data\News_Category_Dataset_new.json


C:\Users\Max\Project\access_data.py:233: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  random_df['category'] = 'NONE'


There were not enough data to create a split with split_ratio 0.3. Only selected 99332 instead of 114380 random data
There were not enough data to create a split with split_ratio 0.3. Only selected 92772 instead of 136247 random data


### creating configuration for label

In [7]:
class ClassifierContainer:
    def __init__(self, label, classifier, vectorizer, svd):
        self.label = label
        self.classifier = classifier
        self.vectorizer = vectorizer
        self.svd = svd
        self.thrashold = 0.5

In [8]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

svc50_list = ["SCIENCE", "SPORTS", "IMPACT"]
svc25_list = ["POLITICS", "LIFESTYLE", "HEALTH", "EDUCATION", "WORLDPOST", "BUSINESS", "RANDOM"]


classifierSVC = {}
classifierLogReg = {}

for key, df in datasets.items():
    #splitting data
    split_map_all = split_dataset(df)
    X_train_all = split_map_all['X_train']
    y_train_all = split_map_all['y_train']
    X_test_all = split_map_all['X_test']
    y_test_all = split_map_all['y_test']

    #encoding label
    integer_encoded_train = y_train_all.replace("NONE", 0).replace(key, 1)
    integer_encoded_test = y_test_all.replace("NONE", 0).replace(key, 1)
#    label_encoder = MyLabelEncoder()
#    label_encoder.fit(["NONE", key])
#    integer_encoded_train = label_encoder.transform(y_train_all)
#    integer_encoded_test = label_encoder.transform(y_test_all)
#    print("0: "+ label_encoder.inverse_transform(0))
#    print("1: "+ label_encoder.inverse_transform(1))    
#    print(y_train_all.unique())
#    print(integer_encoded_test[:20])
 
    tfidf_vectorizer =  TfidfVectorizer(stop_words='english', max_df=0.15, min_df=5)
    X_train = tfidf_vectorizer.fit_transform(X_train_all)
    X_test = tfidf_vectorizer.transform(X_test_all)
    
    # causes problems... would need to save vectorizer
    from sklearn.decomposition import TruncatedSVD
    svd = TruncatedSVD(n_components=2500, n_iter=4, random_state=42)    # there are 52120 features
    X_train_SVD = svd.fit_transform(X_train)
    X_test_SVD = svd.transform(X_test)
    
    if key in svc50_list:
        svc50 = LinearSVC(C=0.5)
        svc50.fit(X_train, integer_encoded_train)
        classifierSVC[key] = ClassifierContainer(key, svc50, tfidf_vectorizer, svd)
    elif key in svc25_list:
        svc25 = LinearSVC(C=0.25)
        svc25.fit(X_train, integer_encoded_train)
        classifierSVC[key] = ClassifierContainer(key, svc25, tfidf_vectorizer, svd)
    
    log_reg = LogisticRegression(C=5)
    log_reg.fit(X_train, integer_encoded_train)
    classifierLogReg[key] = ClassifierContainer(key, log_reg, tfidf_vectorizer, svd)

TRAIN: [ 9975   478 11889 ... 19803 27559 27463] TEST: [19232  8195 25501 ... 14809 26032 23200]
TRAIN: [11105 25912 11690 ... 18426 20577 19001] TEST: [ 4072 14504 23642 ...  9782  3363  4826]
TRAIN: [25957  7494 14539 ... 13265 21465  1473] TEST: [28103 17290 23674 ...   528 10190 15578]
TRAIN: [ 4638  4038  9709 ... 27775  9766  6430] TEST: [18428  8819 13639 ... 23460  5780  6300]
TRAIN: [ 9533  3232  9240 ... 27884  7049 25753] TEST: [22600 14826 25084 ...  7760 10021 21877]
TRAIN: [24984 10558 10995 ...   609  8483 12210] TEST: [23212 22191 20015 ... 19767 24476 14857]
TRAIN: [18493  1877  7421 ... 19491  8961 17558] TEST: [11742   300  6014 ... 18153  8133 19345]
TRAIN: [17165  4837 26025 ... 21576 18177 15205] TEST: [11332  5458  5814 ... 25421 12743  5583]
TRAIN: [15643 13195 13056 ... 12681 17391 18158] TEST: [24815  2544 13635 ...  7318 15689 16061]
TRAIN: [14020  6802   327 ...  9449 20954  9232] TEST: [12509 12223 18107 ...  3411 22723 16605]
TRAIN: [101684 103576 122187 .

# reading in real Data

In [47]:
import os
import pandas as pd

script_dir = os.getcwd()
file_path = os.path.join(script_dir, 'Data\\dataDB10000.csv')

data = pd.read_csv(file_path)

relevant_columns = ['doc_text']
newDf = data[relevant_columns].copy()

In [48]:
newDf['cleaned_text'] = clean_text(newDf['doc_text'])

In [49]:
for key, classifierContainer in classifierSVC.items():   
    tfidf_vectorizer =  classifierContainer.vectorizer
    
    text_cleaned = newDf['cleaned_text']
    text_prepared = tfidf_vectorizer.transform(text_cleaned)
    
    svc = classifierContainer.classifier   
    prediction = svc.predict(text_prepared)
    
    newDf[key+'_svc'] = prediction       

In [50]:
### AUSLAGERN ###

from numpy import array

def convert_probability_list(list):
    dummyList = []
    for l in list:
        dummyList.append(l[-1])
    
    return array(dummyList)

In [52]:
for key, classifierContainer in classifierLogReg.items():   
    tfidf_vectorizer =  classifierContainer.vectorizer
    
    text_cleaned = newDf['cleaned_text']
    text_prepared = tfidf_vectorizer.transform(text_cleaned)
    
    log_reg = classifierContainer.classifier   
    prediction_raw = log_reg.predict_proba(text_prepared)
    prediction = log_reg.predict(text_prepared)
    
#    newDF[key+'_log_RAW'] = convert_probability_list(prediction_raw)
    newDf[key+'_log_reg'] = prediction       

In [24]:
newDf.head()

,doc_text,cleaned_text,SCIENCE_svc,SPORTS_svc,POLITICS_svc,LIFESTYLE_svc,HEALTH_svc,EDUCATION_svc,WORLDPOST_svc,IMPACT_svc,...,EDUCATION_log_RAW,EDUCATION_log_reg,WORLDPOST_log_RAW,WORLDPOST_log_reg,IMPACT_log_RAW,IMPACT_log_reg,BUSINESS_log_RAW,BUSINESS_log_reg,RANDOM_log_RAW,RANDOM_log_reg
0,It seems clear that Niantic always had broader...,it seems clear that niantic always had broader...,1,0,0,0,0,0,0,0,...,0.115269,0,0.007730,0,0.253807,0,0.900786,1,0.014680,0
1,"Attic Labs, the creator of Noms, an open-sourc...",attic the creator of an decentralized announce...,1,0,0,0,0,0,0,0,...,0.292820,0,0.029468,0,0.114623,0,0.639823,1,0.028186,0
2,"Uberall, the Berlin-headquartered startup that...",the startup that ha developed a platform to le...,1,0,0,0,0,0,0,0,...,0.140552,0,0.037446,0,0.157176,0,0.999319,1,0.001039,0
3,"As you might have heard, last year s TC Roboti...",a you might have last year s tc robotics event...,1,0,0,0,0,0,0,1,...,0.100981,0,0.004020,0,0.606501,1,0.644526,1,0.016397,0
4,If you ve tried selling your old smartphone on...,if you ve tried selling your old smartphone on...,1,0,0,0,0,0,0,0,...,0.123017,0,0.003138,0,0.139126,0,0.999933,1,0.018766,0


In [59]:
slicedDF = newDf.iloc[:,2:32].copy() 

,SCIENCE_svc,SPORTS_svc,POLITICS_svc,LIFESTYLE_svc,HEALTH_svc,EDUCATION_svc,WORLDPOST_svc,IMPACT_svc,BUSINESS_svc,RANDOM_svc,SCIENCE_log_reg,SPORTS_log_reg,POLITICS_log_reg,LIFESTYLE_log_reg,HEALTH_log_reg,EDUCATION_log_reg,WORLDPOST_log_reg,IMPACT_log_reg,BUSINESS_log_reg,RANDOM_log_reg
0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0
2,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0
3,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,1,1,0
4,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0
5,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,1,1,0
6,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,1,1,0
7,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0
8,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0


In [61]:
#counting similarities

for key, value in datasets.items():
    same_count = newDf[(newDf[key+'_svc'] == 1) & (newDf[key+'_log_reg'] == 1)].shape[0]
    only_svd = newDf[(newDf[key+'_svc'] == 1)&(newDf[key+'_log_reg'] == 0)].shape[0]
    only_logReg = newDf[(newDf[key+'_svc'] == 0)&(newDf[key+'_log_reg'] == 1)].shape[0]
    print('For label {} there are:'.format(key))
    print('same: {}'.format(same_count))
    print('only svd: {}'.format(only_svd))
    print('only log reg: {}'.format(only_logReg))
    print('##################################################################')

nolabel_count = newDf[slicedDF.sum(axis = 1, skipna = True) < 1].shape[0]
print('There are {} entries without a label'.format(nolabel_count))

For label SCIENCE there are:
same: 4621
only svd: 58
only log reg: 107
##################################################################
For label SPORTS there are:
same: 266
only svd: 18
only log reg: 3
##################################################################
For label POLITICS there are:
same: 2099
only svd: 22
only log reg: 157
##################################################################
For label LIFESTYLE there are:
same: 432
only svd: 29
only log reg: 23
##################################################################
For label HEALTH there are:
same: 203
only svd: 36
only log reg: 8
##################################################################
For label EDUCATION there are:
same: 236
only svd: 0
only log reg: 52
##################################################################
For label WORLDPOST there are:
same: 794
only svd: 32
only log reg: 30
##################################################################
For label IMPACT there are:
same: 1207
onl

In [35]:
same_count = newDf.loc[newDf[key+'_svc'] == 1]

In [79]:
for i in range(0,20):
#    print('--------------------------------{}--------------------------------'.format(newDf['BUSINESS_log_RAW'].iloc[i]))
    print(newDf[slicedDF.sum(axis = 1, skipna = True) < 1]['doc_text'].iloc[i])
    print('##########################################################################')

Two weeks after his company attempted to impose a policy targeted at curbing hate content and hateful conduct, Spotify s CEO admitted the company mishandled its roll out. During an interview at this week s Code Conference, Daniel Ek told the crowd, The whole goal with this was to make sure that we didn t have hate speech on the service. It was never about punishing one individual artist, or even naming one individual artist as well. The policy, introduced on May 10, pulled certain artists from Spotify s curated content streams over bad conduct in their personal lives. Pushback on the policy was almost instantaneous, and reports surfaced last week that Spotify was rethinking its approach. In particular, rapper XXXTencion, who was one of two artists single out by the service (along with R. Kelly), was reportedly going to be added back to Spotify s popular Rap Caviar playlist at some unspecified point. Ek acknowledged that the implementation could have been handled better, and that Spotif

In [82]:
for key, value in datasets.items():
    
    for i in range(0,20):
        print('============================{}============================='.format(key))
        print(newDf[(newDf[key+'_svc'] == 1) & (newDf[key+'_log_reg'] == 1)]['doc_text'].iloc[i])
        print('##################################################################')

============================SCIENCE=============================
It seems clear that Niantic always had broader plans than simple casual gaming. The Google-supported startup s 2016 title Pok mon GO is largely considered a major factor in helping to mainstream augmented reality technology. Now the company is looking to take things a step further, tapping its massive user base to help build some large-scale AR maps. The plan was first reported by Reuters this morning, courtesy of a conversation with Niantic CEO John Hanke. The executive told the service, We want players to build out the game board they want to play on, adding that the world building would foster a new kind of social activity. There s no timeline for any of this. And it s not entirely clear whether the mapping will be output from Pok mon or the company s eagerly anticipated followup, Harry Potter: Wizards Unite. What is clear, however, is that the company plans to start in public spaces like parks, utilizing AR tech built

In [76]:
list(newDf)

['doc_text',
 'cleaned_text',
 'SCIENCE_svc',
 'SPORTS_svc',
 'POLITICS_svc',
 'LIFESTYLE_svc',
 'HEALTH_svc',
 'EDUCATION_svc',
 'WORLDPOST_svc',
 'IMPACT_svc',
 'BUSINESS_svc',
 'RANDOM_svc',
 'SCIENCE_log_reg',
 'SPORTS_log_reg',
 'POLITICS_log_reg',
 'LIFESTYLE_log_reg',
 'HEALTH_log_reg',
 'EDUCATION_log_reg',
 'WORLDPOST_log_reg',
 'IMPACT_log_reg',
 'BUSINESS_log_reg',
 'RANDOM_log_reg']